In [2]:
# Dependencies and Setup
import pandas as pd

# Read CSV Files
school_data = pd.read_csv("Resources/schools_complete.csv")
student_data = pd.read_csv("Resources/students_complete.csv")

# Left join merge both csvs into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


In [158]:
# LGA Summary -> Number of unique schools | Total students + budget | Average maths + reading score | % passing (50%) in reading/maths + overall (passed both)

# Count of unique schools
school_count = len(school_data_complete["school_name"].unique())

# Total students - just counting rows | Created function to format number with comma spacing
def comma_spacing(value):
    return '{:,}'.format(value)
student_count = comma_spacing(school_data_complete["student_name"].count())

# Total budget - sum of budget from each unique school
budget_list = []
schools_budget = list(school_data_complete["budget"])
for i in schools_budget:
    if i not in budget_list:
        budget_list.append(i)

# Create function to automatically convert formatting to currency (dollar sign + 2 decimal places)
def currency(value):
    return "${:,.2f}".format(value)

total_budget = currency(sum(budget_list))

# Average maths score - take maths column and average it | Create function to format to 6 decimal places
def six_decimals(value):
    return '{:.6f}'.format(value)

average_maths = six_decimals(school_data_complete["maths_score"].mean())

# Average reading score - same as above
average_reading = six_decimals(school_data_complete["reading_score"].mean())

# % Passing maths - filter with loc -> get the percentage between counts of new/old * 100
passing_maths = school_data_complete.loc[school_data_complete["maths_score"] >= 50]
passing_maths_percent = six_decimals(passing_maths["maths_score"].count()/school_data_complete["maths_score"].count() * 100)

# % Passing reading - same as above
passing_reading = school_data_complete.loc[school_data_complete["reading_score"] >= 50]
passing_reading_percent = six_decimals(passing_reading["reading_score"].count()/school_data_complete["reading_score"].count() * 100)

# % overall passing - students that passed both math + reading
passing_overall = school_data_complete.loc[(school_data_complete["reading_score"] >= 50) & (school_data_complete["maths_score"] >= 50)]
passing_overall_percent = six_decimals(passing_overall["Student ID"].count()/school_data_complete["Student ID"].count() * 100)

# Create new summary dataframe/table with the above variables - should have 2 rows with 8 columns
summary_list = [
    {
    "Total Schools":school_count,
    "Total Students":student_count,
    "Total Budget":total_budget,
    "Average Maths Score":average_maths,
    "Average Reading Score":average_reading,
    "% Passing Maths":passing_maths_percent,
    "% Passing Reading":passing_reading_percent,
    "% Overall Passing":passing_overall_percent
    }
]

lga_summary = pd.DataFrame(summary_list)
lga_summary


,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


In [157]:
# School Summary - School name + type | total students + budget + per student budget | average reading/maths score | % Passing

# Sort whole dataset by school
groupby_school = school_data_complete.groupby(["school_name"])

# Get type for each school
school_name = groupby_school["school_name"].unique()

# Get type for each school
school_type = groupby_school["type"].unique()
school_type = school_type.astype(str)

# Total students per school
total_student = groupby_school.size()

# Total budget per school
total_school_budget = groupby_school["budget"].unique()
total_school_budget = total_school_budget.astype(int)

# Per student budget
per_student_budget = total_school_budget/total_student

# Average subject scores per school
average_school_reading = groupby_school["reading_score"].mean()
average_school_maths = groupby_school["maths_score"].mean()

# % Pass per school for each subject + overall
pass_school_reading = school_data_complete[school_data_complete["reading_score"] >= 50].groupby(["school_name"]).count()
pass_school_reading = pass_school_reading["reading_score"]/total_student * 100

pass_school_maths = school_data_complete[school_data_complete["maths_score"] >= 50].groupby(["school_name"]).count()
pass_school_maths = pass_school_maths["reading_score"]/total_student * 100

pass_school_overall = school_data_complete[(school_data_complete["maths_score"] >= 50) & (school_data_complete["reading_score"] >= 50)].groupby(["school_name"]).count()
pass_school_overall = pass_school_overall["student_name"]/total_student * 100


# Create a school-grouped summary dataframe with .concat

school_summary = pd.concat([school_type, total_student, total_school_budget, per_student_budget, average_school_maths, average_school_reading, pass_school_maths, pass_school_reading, pass_school_overall],\
axis=1,\
keys=["School Type","Total Students","Total School Budget","Per Student Budget","Average Maths Score","Average Reading Score","% Passing Maths","% Passing Reading","% Overall Passing"])

# print(school_summary)
# print(school_summary.dtypes)

# Formatting summary dataframe
## Remove all square brackets in [School Type] - can't remove these quotation marks for some reason
school_summary["School Type"] = school_summary["School Type"].str.replace("["," ", regex=True).str.replace("]"," ",regex=True)

## Convert [Total School Budget] & [Per Student Budget] into currency format
school_summary[["Total School Budget","Per Student Budget"]] = school_summary[["Total School Budget","Per Student Budget"]].astype("float")

school_summary["Total School Budget"] = school_summary["Total School Budget"].apply(currency)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].apply(currency)

school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,'Government',4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,'Independent',1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,'Government',2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,'Government',2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,'Independent',1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,'Government',4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,'Independent',427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,'Government',2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,'Government',4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
